In [101]:
# imports
import pandas as pd
import numpy as np
import math
import Pyro4
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from nltk.stem.snowball import PorterStemmer
% matplotlib inline
import seaborn as sns
# NLP tools
import nltk
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
import gensim
from gensim.matutils import Sparse2Corpus, corpus2dense
from sklearn.decomposition import PCA
from gensim.corpora.dictionary import Dictionary
from gensim import corpora
from nltk.tokenize import sent_tokenize, word_tokenize
from __future__ import print_function, division
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics.pairwise as smp
from gensim.models import Word2Vec
from sklearn.decomposition import NMF
# Python 2 compatibility
from __future__ import print_function
# logging for gensim (set to INFO)
import logging
from nltk.corpus import stopwords
from gensim.models.coherencemodel import CoherenceModel
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [102]:
questions_train = pd.read_csv("/home/kalgi/train.csv")
question1 = questions_train.iloc[:,4]
question2 = questions_train['question2']
questions_all = pd.concat([question1, question2], axis=0).reset_index()
questions = (questions_all['question2'][:100000]).reset_index()
question_list = list(questions)
result = []
for i in question_list:
    if str(i) != 'nan':
        result.append(i)

In [103]:
# Count Vectorizer & LDA
english_stemmer = PorterStemmer()
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([english_stemmer.stem(w) for w in analyzer(doc)])
cnt = StemmedCountVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", min_df=200, 
                      max_df=0.8, ngram_range=(1,3))
count_vec = cnt.fit(questions['question2'])
counts = count_vec.transform(questions['question2']).transpose()

In [104]:
cnt_corpus = Sparse2Corpus(counts)
id2word_count = dict((v, k) for k, v in cnt.vocabulary_.items())
id2word_count = corpora.Dictionary.from_corpus(cnt_corpus, 
                                         id2word=id2word_count)
lda = models.LdaMulticore(corpus=cnt_corpus, num_topics=12, id2word=id2word_count, passes=50, eval_every=10, workers=8)
lda_corpus = lda[cnt_corpus]
#topic_probs = list(lda_corpus)
#questions['topic_probs'] = pd.Series(topic_probs)
#questions['topic_probs'] = questions['topic_probs'].apply(lambda row: sorted(row, key=lambda x: x[1], reverse=True))
#questions.to_csv('/dev/qt.csv')
lda_docs = [doc for doc in lda_corpus]


In [105]:
cm = CoherenceModel(model=lda, corpus=cnt_corpus, dictionary=id2word, coherence='u_mass')
cm.get_coherence()

-8.5877509806920713

In [89]:
questions['lda_vecs'] = lda_docs
lda.print_topics(num_words=30)

[(0,
  '0.055*"english" + 0.045*"weight" + 0.041*"lose" + 0.041*"improv" + 0.038*"place" + 0.035*"comput" + 0.032*"scienc" + 0.030*"data" + 0.028*"song" + 0.026*"men" + 0.025*"china" + 0.025*"idea" + 0.024*"visit" + 0.023*"women" + 0.022*"hair" + 0.022*"consid" + 0.021*"skill" + 0.020*"rid" + 0.020*"lose weight" + 0.019*"histori" + 0.019*"speak" + 0.018*"big" + 0.018*"citi" + 0.018*"plan" + 0.017*"option" + 0.017*"form" + 0.016*"startup" + 0.015*"bodi" + 0.015*"gain" + 0.014*"commun"'),
 (1,
  '0.108*"like" + 0.104*"make" + 0.065*"money" + 0.043*"girl" + 0.039*"don" + 0.032*"onlin" + 0.031*"friend" + 0.031*"want" + 0.026*"peopl" + 0.026*"sex" + 0.023*"guy" + 0.023*"realli" + 0.021*"earn" + 0.020*"look" + 0.018*"know" + 0.018*"market" + 0.017*"tell" + 0.016*"date" + 0.016*"make money" + 0.015*"exist" + 0.015*"mind" + 0.013*"control" + 0.013*"god" + 0.013*"man" + 0.012*"talk" + 0.012*"file" + 0.011*"digit" + 0.011*"earn money" + 0.011*"money onlin" + 0.010*"feel"'),
 (2,
  '0.078*"time" 

In [100]:
index = similarities.MatrixSimilarity(questions['lda_vecs'],
                                      num_features=300)
sorted_sims = sorted(enumerate(index[questions['lda_vecs'][10000]]), key=lambda item: -item[1])
print(sorted_sims[0:10])
print(questions['question2'][10000])
print(questions['question2'][6686])
print(questions['question2'][12432])
print(questions['question2'][19189])
print(questions['question2'][22246])
print(questions['question2'][29741])
print(questions['question2'][30491])

[(6686, 1.0), (12432, 1.0), (19189, 1.0), (22246, 1.0), (29741, 1.0), (30491, 1.0), (30558, 1.0), (31884, 1.0), (32465, 1.0), (37995, 1.0)]
Is it safe for a woman to travel alone in Japan?
What purpose does the nuclear membrane serve? What are its functions?
When and where did the term "venture capital" come to being?
How do jet engines actually work?
I got a package of 3.3 lakh per annum. What is my monthly salary after all the calculations?
What's your feeling about the Chinese films?
How much power does my 40W lamp consume in 1 hour?


What keeps childern active and far from phone and video games? 7, 9, 0
Where can I host a Java website for free like 000WebHost? 7, 4
Has perpetual motion or any free energy generator been achieved? If no, is it even possible to achieve it? If yes, how close are we to achieve it? 2,4,9
If you view a video on someone's timeline that you aren't friends with can they see you viewed it? 5,7
Friends (TV series): What would have changed if Matt LeBlanc played Chandler Bing? 4,8
How do you track down a person if you know their name and the rough area of where they live? 8,7,9
What will you do if you regret your choices later and now you can not change it and have to live with it for whole life? 8,7,5
